In [1]:
import pandas as pd
import nflgame
from pprint import pprint
from collections import defaultdict
import pickle

In [2]:
CURR_WEEK = 14

In [3]:
YEARS = [2013,2014,2015]

In [36]:
games = nflgame.games(2013, 1)
game_player_stats = nflgame.combine_game_stats(games[:1])
playerstats = defaultdict(str)

In [21]:
game1 = games[0]

In [39]:
for p in game_player_stats:
    print(p.name, p.guess_position)

(u'P.Manning', '')
(u'K.Moreno', '')
(u'M.Ball', '')
(u'R.Hillman', '')
(u'W.Welker', '')
(u'J.Thomas', u'TE')
(u'A.Caldwell', '')
(u'D.Thomas', u'WR')
(u'E.Decker', u'WR')
(u'D.Trevathan', u'ILB')
(u'M.Prater', u'K')
(u'B.Colquitt', u'P')
(u'T.Holliday', '')
(u'T.Knighton', '')
(u'R.Moore', '')
(u'K.Webster', u'CB')
(u'D.Wolfe', u'DE')
(u'M.Adams', u'SS')
(u'P.Lenon', '')
(u'S.Phillips', '')
(u'C.Harris', u'CB')
(u'R.Ayers', u'DE')
(u'K.Vickerson', '')
(u'D.Ihenacho', '')
(u'M.Unrein', u'DE')
(u'T.Carter', '')
(u'S.Williams', u'NT')
(u'W.Woodyard', u'ILB')
(u'N.Irving', '')
(u'M.Jackson', u'DT')
(u'J.Flacco', u'QB')
(u'R.Rice', '')
(u'B.Pierce', '')
(u'J.Jones', '')
(u'D.Clark', '')
(u'M.Brown', '')
(u'E.Dickson', u'TE')
(u'T.Smith', u'WR')
(u'V.Leach', '')
(u'B.Stokley', '')
(u'R.Wagner', u'OT')
(u'M.Cox', u'LS')
(u'J.Tucker', u'K')
(u'S.Koch', u'P')
(u'L.Webb', u'FS')
(u'D.Smith', '')
(u'C.Canty', '')
(u'H.Ngata', u'DT')
(u'E.Dumervil', u'DE')
(u'M.Huff', '')
(u'C.Graham', u'DB')
(u

In [3]:
for p in game_player_stats:
    info = {}
    info.update(p.stats)
    try:
        info.update(p.player.__dict__)
    except AttributeError:
        pass
    playerstats[p.playerid] = info
tmp = pd.DataFrame(playerstats).T.reset_index().rename(columns={"index":"id"})

In [29]:
def get_year(year, weeks):
    """ Get all games and players involved for a given season"""
    week_data = []
    for week in weeks:
        games = nflgame.games(year, week)
        game_player_stats = nflgame.combine_game_stats(games)
        df = get_week(game_player_stats, week)
        df['week'] = week
        week_data.append(df)
    return pd.concat(week_data)

def get_week(players, week):
    """ Get stats for each player in each game"""
    playerstats = defaultdict(str)
    for p in players:
        info = {}
        info.update(p.stats)
        try:
            info.update(p.player.__dict__)
        except AttributeError:
            pass
        info['team'] = p.team
        playerstats[p.playerid] = info
    return pd.DataFrame(playerstats).T.reset_index().rename(columns={"index":"id"})

def get_opponent_stats(team_list, year):
    """ For a given team, report opponents' stats in each game to get measures of
        the team's defensive stats"""
    dfs = []
    for team_to_check in team_list:
        print team_to_check
        # Get the games the target team played in the season
        games = nflgame.games(year, home=team_to_check, away=team_to_check)
        rows = []
        for g in games:
            row = {}
            if g.home == team_to_check:
                row['week'] = g.schedule['week']
                row['TEAM'] = team_to_check
                row['OPP'] = g.away
                row['opp_points'] = g.score_away
                row.update(g.stats_away.__dict__)
                row['pos_time'] = row['pos_time'].total_seconds()
            else:
                row['week'] = g.schedule['week']
                row['TEAM'] = team_to_check
                row['OPP'] = g.home
                row['opp_points'] = g.score_home
                row.update(g.stats_home.__dict__)
                row['pos_time'] = row['pos_time'].total_seconds()

            rows.append(row)
        dfs.append(pd.DataFrame(rows))
    return pd.concat(dfs)

In [33]:
TEAMS = [team[0] for team in nflgame.teams] +['STL']
TEAMS.remove('LA')

In [34]:
weeks = list(range(1,18))

In [35]:
ps = []
for year in YEARS:
    print "Creating Players Stats "+str(year)
    player_stats = get_year(year, weeks).fillna(0)
    ps.append(player_stats)
    print "Creating Opponents Stats "+str(year)
    opp_stats = get_opponent_stats(TEAMS, year)
    opp_stats = opp_stats.add_prefix("opp_")
    print "Saving to File "+str(year)
    opp_stats.to_csv("./data/opp_stats_"+str(year)+".csv")
    player_stats.to_csv("./data/player_stats_"+str(year)+".csv")

Creating Players Stats 2013
Creating Opponents Stats 2013
ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
JAC
KC
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SD
SEA
SF
STL
TB
TEN
WAS
STL
Saving to File 2013
Creating Players Stats 2014
Creating Opponents Stats 2014
ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
JAC
KC
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SD
SEA
SF
STL
TB
TEN
WAS
STL
Saving to File 2014
Creating Players Stats 2015
Creating Opponents Stats 2015
ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
JAC
KC
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SD
SEA
SF
STL
TB
TEN
WAS
STL
Saving to File 2015


In [40]:
TEAMS_2016 = [team[0] for team in nflgame.teams]
TEAMS_2016.remove('JAC')
TEAMS_2016.append('JAX')
TEAMS_2016.remove('STL')

year = 2016
print "Creating Players Stats "+str(year)
player_stats = get_year(year, weeks).fillna(0)
print "Creating Opponents Stats "+str(year)
opp_stats = get_opponent_stats(TEAMS_2016, year)
opp_stats = opp_stats.add_prefix("opp_")
opp_stats.to_csv("./data/opp_stats_"+str(year)+".csv")
player_stats.to_csv("./data/player_stats_"+str(year)+".csv")

Creating Players Stats 2016
Creating Opponents Stats 2016
ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
KC
LA
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SD
SEA
SF
TB
TEN
WAS
JAX


In [41]:
TEAMS_2017 = [team[0] for team in nflgame.teams]
TEAMS_2017.remove('JAC')
TEAMS_2017.append('JAX')
TEAMS_2017.remove('STL')
TEAMS_2017.append('LAC')
TEAMS_2017.remove('SD')

weeks = list(range(1,CURR_WEEK))
year = 2017
print "Creating Players Stats "+str(year)
player_stats = get_year(year, weeks).fillna(0)
print "Creating Opponents Stats "+str(year)
opp_stats = get_opponent_stats(TEAMS_2017, year)
opp_stats = opp_stats.add_prefix("opp_")
opp_stats.to_csv("./data/opp_stats_"+str(year)+".csv")
player_stats.to_csv("./data/player_stats_"+str(year)+".csv")

Creating Players Stats 2017
Creating Opponents Stats 2017
ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
KC
LA
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SEA
SF
TB
TEN
WAS
JAX
LAC
